In [1]:
%run algebra.py
%run cache.py
%run costs.py
%run features.py
%run gradients.py
%run helpers.py
%run model.py
%run models.py
%run splits.py

import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import csv
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
SUB_SAMPLE = True
CACHE_DIR = "test/cache/" if SUB_SAMPLE else "cache/"
SUBMISSIONS_DIR = "test/submissions/" if SUB_SAMPLE else "submissions/"

In [3]:
def load_csv_data(data_path, sub_sample=True):
    """Loads data and returns y (class labels), tX (features) and ids (event ids)"""
    y = np.genfromtxt(data_path, delimiter=",", skip_header=1, dtype=str, usecols=1)
    x = np.genfromtxt(data_path, delimiter=",", skip_header=1)
    ids = x[:, 0].astype(np.int)
    input_data = x[:, 2:]

    # convert class labels from strings to binary (-1,1)
    yb = np.ones(len(y))
    yb[np.where(y=='b')] = -1

    # sub-sample
    if sub_sample:
        yb = yb[::50]
        input_data = input_data[::50]
        ids = ids[::50]

    return yb, input_data, ids

In [4]:
y, x, ids = load_csv_data('data/train.csv', SUB_SAMPLE)

### Logistic Regression

In [5]:
class First_Order_Logistic_Regression_Model2(Model):

    def prepare(self, x, y, h):
        
        degree = int(h['degree'])

        x = remove_errors(x)
        x = remove_outliers(x)
        x = standardize_all(x)
        x = remove_nan_features(x)
        x = build_poly(x, degree)
        
        return x, y

    def fit(self, x, y, h={}):
        
        batch_size = int(h['batch_size'])
        n_iters = int(h['n_iters'])
        gamma = float(h['gamma'])
        
        initial_w = np.zeros(x.shape[1])
        return logistic_regression(y, x, initial_w, batch_size, n_iters, gamma)
    
    def test(self, x, y, w, h):
        mse = compute_mse(y, x, w)
        if np.isnan(mse):
            mse = np.inf
        return { 'mse': mse }

In [17]:
myModel = CrossValidationModel(First_Order_Logistic_Regression_Model2())

n_iters = [1000]
batch_size = [1]
degrees = np.arange(15)
gammas = np.logspace(-15, -10, 10)

hs={
    'n_iters': n_iters,
    'batch_size': batch_size,
    'degree': degrees,
    'gamma': gammas,
    'k_fold': [4],
    'seed': [0]
}

res = myModel.evaluate(x, y, hs, filename=CACHE_DIR+'Logistic_Regression_Explo')

#print(res)

#plot_heatmap(res, hs, 'mse_te', 'degree', 'gamma')

#res_mse = np.vectorize(lambda x: x['mse'])(res)
#x_axis = np.unique(np.vectorize(lambda x: x['gamma'])(res))
#y_axis = np.unique(np.vectorize(lambda x: x['degree'])(res))

#plot_heatmap(res, hs, 'mse', 'degree', 'gamma')
find_arg_min(res, 'mse_te')

(array([126]),)


{'n_iters': 1000,
 'batch_size': 1,
 'degree': 6,
 'gamma': 2.7825594022071258e-11,
 'k_fold': 4,
 'seed': 0,
 'mse_tr': 0.49696297914908244,
 'mse_te': 0.49791066042234666}

In [ ]:
hs = {'batch_size': 10.0,
 'degree': 1.0,
 'gamma': 0.0003548133892335753,
 'k_fold': 4.0,
 'n_iters': 100000.0,
 'seed': 0.0,
 'mse_te': 0.3988701335328735,
 'mse_tr': 0.3988200000042866}

myModel.predict(hs, x, y, SUBMISSIONS_DIR + 'Logistic_Regression')

### Reg Logistic Regression

In [9]:
class Reg_Logistic_Regression_Model2(Model):

    def prepare(self, x, y, h):
        
        degree = int(h['degree'])

        x = remove_errors(x)
        x = remove_outliers(x)
        x = standardize_all(x)
        x = remove_nan_features(x)
        x = build_poly(x, degree)
        
        return x, y

    def fit(self, x, y, h={}):
        
        batch_size = int(h['batch_size'])
        n_iters = int(h['n_iters'])
        gamma = float(h['gamma'])
        lambda_ = float(h['lambda'])
        
        initial_w = np.zeros(x.shape[1])
        return reg_logistic_regression(y, x, initial_w, batch_size, n_iters, gamma, lambda_)
    
    def test(self, x, y, w, h):
        mse = compute_mse(y, x, w)
        if np.isnan(mse):
            mse = np.inf
        return { 'mse': mse }

In [7]:
myModel = CrossValidationModel(Reg_Logistic_Regression_Model2())

n_iters = [1000]
batch_size = [1]
degrees = np.arange(5,15)
gammas = np.logspace(-15, -10, 10)
lambdas = np.logspace(-10, 0, 3)

hs={
    'n_iters': n_iters,
    'batch_size': batch_size,
    'degree': degrees,
    'gamma': gammas,
    'lambda': lambdas,
    'k_fold': [4],
    'seed': [0]
}

res = myModel.evaluate(x, y, hs, filename=CACHE_DIR+'Reg_Logistic_Regression_Explo')

print(res)

#plot_heatmap(res, hs, 'mse_te', 'degree', 'gamma')

#res_mse = np.vectorize(lambda x: x['mse'])(res)
#x_axis = np.unique(np.vectorize(lambda x: x['gamma'])(res))
#y_axis = np.unique(np.vectorize(lambda x: x['degree'])(res))

#plot_heatmap(res, hs, 'mse', 'degree', 'gamma')
find_arg_min(res, 'mse_te')

[{'batch_size': 1.0, 'degree': 5.0, 'gamma': 1e-15, 'k_fold': 4.0, 'lambda': 1e-10, 'n_iters': 1000.0, 'seed': 0.0, 'mse_te': 0.49999999320766625, 'mse_tr': 0.4999999915820104}, {'batch_size': 1.0, 'degree': 6.0, 'gamma': 1e-15, 'k_fold': 4.0, 'lambda': 1e-10, 'n_iters': 1000.0, 'seed': 0.0, 'mse_te': 0.4999998257003512, 'mse_tr': 0.4999997606510462}, {'batch_size': 1.0, 'degree': 7.0, 'gamma': 1e-15, 'k_fold': 4.0, 'lambda': 1e-10, 'n_iters': 1000.0, 'seed': 0.0, 'mse_te': 0.4999953335812706, 'mse_tr': 0.4999925425399327}, {'batch_size': 1.0, 'degree': 8.0, 'gamma': 1e-15, 'k_fold': 4.0, 'lambda': 1e-10, 'n_iters': 1000.0, 'seed': 0.0, 'mse_te': 0.4998840460029312, 'mse_tr': 0.4997610510534121}, {'batch_size': 1.0, 'degree': 9.0, 'gamma': 1e-15, 'k_fold': 4.0, 'lambda': 1e-10, 'n_iters': 1000.0, 'seed': 0.0, 'mse_te': 0.5138117047071616, 'mse_tr': 0.5140159362387455}, {'batch_size': 1.0, 'degree': 10.0, 'gamma': 1e-15, 'k_fold': 4.0, 'lambda': 1e-10, 'n_iters': 1000.0, 'seed': 0.0, 'm

{'batch_size': 1.0,
 'degree': 6.0,
 'gamma': 2.7825594022071258e-11,
 'k_fold': 4.0,
 'lambda': 1.0,
 'n_iters': 1000.0,
 'seed': 0.0,
 'mse_te': 0.4979106604043108,
 'mse_tr': 0.49696297913361}